In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EJ5C46HAM to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141813
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-141813


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "udacity-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Assigning existing cluster...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=0, max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    print('Create compute cluster...')

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())


Create compute cluster...
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-04-02T12:58:09.276000+00:00', 'errors': None, 'creationTime': '2021-04-02T12:58:06.661264+00:00', 'modifiedTime': '2021-04-02T12:58:22.150628+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C' : uniform(0.5, 1.5),
        '--max_iter': choice(50, 100, 150, 200)
    }
)
# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
est = SKLearn(source_directory = os.path.join( "./" ),
        entry_script="train.py",
        compute_target=compute_target
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        estimator=est,
        policy=policy,
        max_total_runs=16,
        max_concurrent_runs=4
    )


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1ad9cd7a-2e6d-4a61-a18a-13e6bfee9e60
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1ad9cd7a-2e6d-4a61-a18a-13e6bfee9e60?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-141813/workspaces/quick-starts-ws-141813

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-02T12:58:47.319434][API][INFO]Experiment created<END>\n""<START>[2021-04-02T12:58:47.916084][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-02T12:58:48.079194][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-02T12:58:49.5134674Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_1ad9cd7a-2e6d-4a61-a18a-13e6bfee9e60
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1ad9cd7a-2e6d-4a61-a18a-13e6bfee9e60?wsid=/subscriptions/510b9

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metrics:", best_run.get_metrics())
print("Best run details:", best_run.get_details())
print("Best run file names:", best_run.get_file_names())



Best run metrics: {'Regularization Strength:': 1.1268045674741702, 'Max iterations:': 200, 'Accuracy': 0.9072837632776934}
Best run details: {'runId': 'HD_1ad9cd7a-2e6d-4a61-a18a-13e6bfee9e60_0', 'target': 'udacity-cluster', 'status': 'Completed', 'startTimeUtc': '2021-04-02T13:02:41.137555Z', 'endTimeUtc': '2021-04-02T13:04:27.044878Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '52b59263-7b1c-43e0-a9a7-673f3414fdf9', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1.1268045674741702', '--max_iter', '200'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'udacity-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': No

In [7]:
# Save the best model
joblib.dump(value=best_run.get_details()['runDefinition']['arguments'], filename=os.path.join('./outputs', 'best_model.joblib'))


['./outputs/best_model.joblib']

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [9]:
from azureml.core import Dataset
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)
x['y'] = y
os.mkdir("./data")
x.to_csv('./data/ds_cleaned.csv')
datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')
ds_cleaned = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/ds_cleaned.csv'))])

Uploading an estimated of 1 files
Uploading data/ds_cleaned.csv
Uploaded data/ds_cleaned.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds_cleaned,
    label_column_name='y',
    compute_target=compute_target,
    n_cross_validations=5)

In [11]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = True)
remote_run.wait_for_completion()


Running on remote.
No run_configuration provided, running on udacity-cluster with default configuration
Running on remote compute: udacity-cluster
Parent Run ID: AutoML_5fe8b0b2-52e0-4b6a-9af6-ae5ccae89734

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because t

{'runId': 'AutoML_5fe8b0b2-52e0-4b6a-9af6-ae5ccae89734',
 'target': 'udacity-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-02T13:11:56.088552Z',
 'endTimeUtc': '2021-04-02T13:53:35.060397Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-141813","workspace_name":"quick-starts-ws-141813","region":"southcentralus","compute_target":"udacity-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":

In [15]:
# Retrieve and save best automl model.
#from azureml.automl.runtime.onnx_convert import OnnxConverter
#best_aml_model, onnx_model = remote_run.get_output(return_onnx_model=True)
#OnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")

### YOUR CODE HERE ###
best_run, best_model = remote_run.get_output()
print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs')
registered_model.download(target_dir='./outputs', exist_ok=True)


In [16]:
# Delete Compute Cluster
compute_target.delete()


Current provisioning state of AmlCompute is "Deleting"

